In [180]:
import numpy as np
import pandas as pd
import plotly as plt
import matplotlib
import seaborn as sns


pd.set_option('display.max_columns', None)


In [181]:

store_df = pd.read_csv('store.csv',low_memory=False)
test_df = pd.read_csv('test.csv',low_memory=False)
train_df = pd.read_csv('train.csv',low_memory=False)


In [182]:
merged_df = pd.merge(train_df,store_df,how='left',on='Store')
merged_df_test = test_df.merge(store_df,how='left',on='Store')

In [183]:
merged_df['Date'].min() , merged_df['Date'].max() , merged_df_test['Date'].min(),merged_df_test['Date'].max()

('2013-01-01', '2015-07-31', '2015-08-01', '2015-09-17')

In [184]:
def split_date(df):
    df['Date']=pd.to_datetime(df['Date'])
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['WeekOfYear'] = df['Date'].dt.isocalendar().week
    
split_date(merged_df)
split_date(merged_df_test)

In [185]:
merged_df[merged_df['Open']==0]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day,WeekOfYear
291,292,5,2015-07-31,0,0,0,1,0,1,a,a,1100.0,6.0,2009.0,0,NaN,NaN,NaN,2015,7,31,31
875,876,5,2015-07-31,0,0,0,1,0,1,a,a,21790.0,4.0,2005.0,1,18.0,2015.0,"Feb,May,Aug,Nov",2015,7,31,31
1406,292,4,2015-07-30,0,0,0,1,0,1,a,a,1100.0,6.0,2009.0,0,NaN,NaN,NaN,2015,7,30,31
1990,876,4,2015-07-30,0,0,0,1,0,1,a,a,21790.0,4.0,2005.0,1,18.0,2015.0,"Feb,May,Aug,Nov",2015,7,30,31
2521,292,3,2015-07-29,0,0,0,1,0,1,a,a,1100.0,6.0,2009.0,0,NaN,NaN,NaN,2015,7,29,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct",2013,1,1,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN,2013,1,1,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN,2013,1,1,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN,2013,1,1,1


In [186]:
merged_df =merged_df[merged_df['Open']==1].copy()

In [187]:
def comp_months(df):
    df['CompetitionOpen'] = 12 * (df['Year'] - df['CompetitionOpenSinceYear']) + df['CompetitionOpenSinceMonth']
    df['CompetitionOpen'] = df['CompetitionOpen'].map(lambda x: 0 if x <0 else x).fillna(0)

In [188]:
comp_months(merged_df)
comp_months(merged_df_test)
merged_df.CompetitionOpen.value_counts()

CompetitionOpen
0.0       312024
33.0       14404
21.0       14201
45.0       10698
69.0        8693
           ...  
204.0        175
211.0        174
197.0        174
1381.0       174
1369.0       147
Name: count, Length: 207, dtype: int64

In [189]:
def check_promo_month(row):
    month2str = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sept',10:'Oct',11:'Nov',12:'Dec'}
    
    try: 
        months= (row['PromoInterval'] or '').split()
        if row['Promo2Open'] and month2str[row['Month']] in months:
            return 1
        else: 
            return 0
    except Exception:
        return 0 

In [ ]:
def promo_cols(df):
    df['Promo2Open'] = 12 * (df.Year - df.Promo2SinceYear) + (df.WeekOfYear-df.Promo2SinceWeek)/4.25
    df['Promo2Open'] = df['Promo2Open'].map(lambda x: 0 if x<0 else x).fillna(0) * df['Promo2']
    df['IsPromo2Month'] = df.apply(check_promo_month,axis=1) * df['Promo2']

In [198]:

promo_cols(merged_df)
promo_cols(merged_df_test)

In [201]:
merged_df.IsPromo2Month.value_counts()

IsPromo2Month
0    844392
Name: count, dtype: int64

In [193]:
merged_df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day,WeekOfYear,CompetitionOpen,Promo2Open,IsPromo2Month
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015,7,31,31,93.0,0.000000,0
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2015,7,31,31,107.0,65.411765,0
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",2015,7,31,31,120.0,52.941176,0
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN,2015,7,31,31,81.0,0.000000,0
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN,2015,7,31,31,4.0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016776,682,2,2013-01-01,3375,566,1,0,a,1,b,a,150.0,9.0,2006.0,0,NaN,NaN,NaN,2013,1,1,1,93.0,0.000000,0
1016827,733,2,2013-01-01,10765,2377,1,0,a,1,b,b,860.0,10.0,1999.0,0,NaN,NaN,NaN,2013,1,1,1,178.0,0.000000,0
1016863,769,2,2013-01-01,5035,1248,1,0,a,1,b,b,840.0,NaN,NaN,1,48.0,2012.0,"Jan,Apr,Jul,Oct",2013,1,1,1,0.0,1.176471,0
1017042,948,2,2013-01-01,4491,1039,1,0,a,1,b,b,1430.0,NaN,NaN,0,NaN,NaN,NaN,2013,1,1,1,0.0,0.000000,0
